<a href="https://colab.research.google.com/github/okcza515/Stock-Prediction/blob/main/Stock_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [125]:
##uncomment the following one, when you need to update something in this cell
%%writefile app.py

import streamlit as st
from datetime import date

import yfinance as yf
from prophet import Prophet
from prophet.plot import plot_plotly
from plotly import graph_objs as go
from pyngrok import ngrok
import time


start = "2015-01-01"
today = date.today().strftime("%Y-%m-%d")

st.title("Stock Prediction App")
st.subheader("By Nutchapong Pramualsap")

selected_stock = ""
selected_stock = st.text_input("Input stock symbol for prediction")
def get_stock_symbols():
    # Use the Ticker class from yfinance to get a list of all available symbols
    all_tickers = yf.Tickers('^GSPC')  # Using S&P 500 as a proxy to get a list of all symbols
    symbols = all_tickers.tickers

    return [symbol.info['symbol'] for symbol in symbols]

st.title("Stock Symbol Viewer")

# Get stock symbols
stock_symbols = get_stock_symbols()

# Display stock symbols
st.write("Available Stock Symbols:")
for symbol in stock_symbols:
  st.write(symbol)

while(selected_stock == ""):
  #wait loop
  time.sleep(0.5)

n_years = st.slider("Years of prediction:", 1, 4)
period = n_years * 365


@st.cache_data
def load_data(stock):
  data = yf.download(stock, start, today)
  data.reset_index(inplace =True)
  return data

data_load_state = st.text("Load data...")
data = load_data(selected_stock)
data_load_state.text("Loading data completed")

st.subheader('Raw data')
st.write(data.tail())

def plot_raw_data():
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=data['Date'],y=data['Open'],name = 'stock_open', line =dict(color = 'green')))
  fig.add_trace(go.Scatter(x=data['Date'],y=data['Close'],name = 'stock_close', line=dict(color= 'red')))
  fig.layout.update(title_text = "Time Series Data", xaxis_rangeslider_visible = True)
  st.plotly_chart(fig)

plot_raw_data()

df_train = data[['Date','Close']]
df_train = df_train.rename(columns={'Date':'ds', 'Close':'y'})

model = Prophet()
model.fit(df_train)
future = model.make_future_dataframe(periods= period)
forecast = model.predict(future)

st.subheader("Forecast data")
st.write(forecast.tail())

fig1 = plot_plotly(model,forecast)
st.plotly_chart(fig1)

st.write('forecast components')
fig2 = model.plot_components(forecast)
st.write(fig2)

Overwriting app.py


In [44]:
#link ngrok on google colab to my ngrok account
ngrok.set_auth_token("2a7m8MP42tnl1fJ39KzEMiCRUbg_3MCoMShMjPUy1mxjuKZFY")

In [73]:
!nohup streamlit run app.py --server.port 80 &
url = ngrok.connect(80)
print(url)

nohup: appending output to 'nohup.out'
NgrokTunnel: "https://5e3c-34-125-133-207.ngrok-free.app" -> "http://localhost:80"
